#### Using gender_guesser and transliteration to check proportion on male & female names in Ukrainian text

In [1]:
import gender_guesser.detector as gender

Let's check the correctness of gender detection.

In [2]:
d = gender.Detector()
d.get_gender("Eugeniya")

'female'

Sometimes it's correct.

In [3]:
d.get_gender("Oleh")

'male'

In [4]:
d.get_gender("Luydmila")

'unknown'

But sometimes it's not. So after autodetection you can check it manually.

In [5]:
from requests import get
from bs4 import BeautifulSoup

Let's collect the list of current Ukrainian Parliament MPs full names.

In [6]:
r=get("http://w1.c1.rada.gov.ua/pls/site2/fetch_mps?skl_id=10")
soup=BeautifulSoup(r.text, "html.parser")
fullnames=[i.text for i in soup.select("p.title")]
fullnames[:10]

['\nАбдуллін Олександр Рафкатович\n',
 '\nАбрамович Ігор Олександрович\n',
 '\nАлєксєєв Сергій Олегович\n',
 '\nАліксійчук Олександр Васильович\n',
 '\nАллахвердієва Ірина Валеріївна\n',
 '\nАнанченко Михайло Олегович\n',
 '\nАндрійович Зіновій Мирославович\n',
 '\nАрахамія Давид Георгійович\n',
 '\nАрешонков Володимир Юрійович\n',
 "\nАр'єв Володимир Ігорович\n"]

Now we'll extract a first name part from each full name.

In [7]:
names=[i.split(" ")[1] for i in fullnames]
names[:10]

['Олександр',
 'Ігор',
 'Сергій',
 'Олександр',
 'Ірина',
 'Михайло',
 'Зіновій',
 'Давид',
 'Володимир',
 'Володимир']

In [8]:
from transliterate import translit

Transliterate each name to latin, as gender_guesser library does not work with Cyrillic.

In [9]:
engnames=[translit(i, "uk", reversed=True) for i in names]
engnames[:10]

['Oleksandr',
 'Ihor',
 'Serhij',
 'Oleksandr',
 'Iryna',
 'Mykhajlo',
 'Zinovij',
 'Davyd',
 'Volodymyr',
 'Volodymyr']

Now autodetect each name gender and see which names remained undetected.

In [10]:
genders=[d.get_gender(i) for i in engnames]
names_genders=list(zip(names,genders))
unknown=[i[0] for i in names_genders if i[1]=="unknown"]
unknown

['Зіновій',
 "Мар'яна",
 'Соломія',
 'Єлізавета',
 'Костянтин',
 'Євгеній',
 'Людмила',
 'Ольга',
 'Галина',
 "Дар'я",
 'Олесь',
 "Мар'ян",
 'Костянтин',
 'Ілля',
 'Ольга',
 'Євгенія',
 'Гео',
 'Георгій',
 'Григорій',
 'Суто',
 'Людмила',
 "В'ячеслав",
 'Галина',
 'Василь',
 'Григорій',
 'Кирилл',
 'Василь',
 'Євген',
 'Василь',
 'Євген',
 'Роксолана',
 'Ольга',
 'Ольга',
 'Ольга',
 'Ольга',
 'Ольга',
 'Григорій',
 'Галина',
 'Єгор',
 'Ахтем',
 'Євгеній',
 'Остап',
 'Альона',
 'Євген',
 'Галина',
 'Єлизавета']

Specify undetected female names. Other undetected names will be marked as "male".

In [11]:
femalenames=["Ольга","Людмила","Мар'яна",'Соломія', 'Єлізавета','Галина', "Дар'я",
             'Євгенія','Роксолана','Альона','Єлизавета']
names_genders=[(i[0],"female") if i[0] in femalenames 
               else (i[0],"male") if i[1]=="unknown" 
else (i[0],i[1]) for i in names_genders]
correctgenders=[i[1] for i in names_genders]

Now we can see the percentage of male and female MPs.

In [12]:
print(f"Total MPs: {len(correctgenders)}")
print(f"Female MPs: {correctgenders.count('female')} ({(correctgenders.count('female')/len(correctgenders))*100} %)")
print(f"Male MPs: {correctgenders.count('male')} ({(correctgenders.count('male')/len(correctgenders))*100} %)")


Total MPs: 423
Female MPs: 87 (20.56737588652482 %)
Male MPs: 336 (79.43262411347519 %)
